# Examine `.pz` File Data

A Jupyter notebook to load a `.pz` file and examine the `usgs_test_original` array.

## Import Required Libraries
Import libraries such as NumPy and Matplotlib for file loading and visualization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os # Import os to construct file paths reliably

## Load the `.pz` File
Use NumPy's load function to load the test `.pz` file and extract its contents. Make sure the `.pz` file is in the same directory or provide the correct path.

In [ ]:
# Define the path to the .pz file
# IMPORTANT: Replace 'your_test_file.pz' with the actual filename.
file_name = 'data/processed/test/20380357.npz' # Example filename, change if needed
file_path = os.path.join('.', file_name) # Assumes file is in the same directory as the notebook

# Check if the file exists before attempting to load
if os.path.exists(file_path):
    # Load the .pz file
    try:
        loaded_data = np.load(file_path, allow_pickle=True)
        print(f"Successfully loaded '{file_name}'.")
        # List the arrays contained within the file
        print("Arrays in file:", list(loaded_data.keys()))
    except Exception as e:
        print(f"Error loading file '{file_name}': {e}")
        loaded_data = None
else:
    print(f"Error: File not found at '{file_path}'")
    loaded_data = None

## Examine the `usgs_test_original` Array
Access the `usgs_test_original` array from the loaded file and print its shape, data type, and a sample of its values.

In [ ]:
# Check if data was loaded successfully and the key exists
if loaded_data is not None and 'usgs_test_original' in loaded_data:
    # Access the array
    usgs_test_original = loaded_data['usgs_test_original']

    # Print shape and data type
    print("Shape of 'usgs_test_original':", usgs_test_original.shape)
    print("Data type of 'usgs_test_original':", usgs_test_original.dtype)

    # Print a sample of the values (e.g., the first 5 elements/rows)
    print("\nSample values (first 5 elements/rows):")
    # Adjust slicing based on the array's dimensions
    if usgs_test_original.ndim == 1:
        print(usgs_test_original[:5])
    elif usgs_test_original.ndim >= 2:
        print(usgs_test_original[:5, ...]) # Print first 5 rows, all columns
    else:
        print(usgs_test_original) # Print the whole array if 0-dim

    # Close the file handle
    loaded_data.close()
elif loaded_data is not None:
    print("Error: 'usgs_test_original' key not found in the loaded file.")
    print("Available keys:", list(loaded_data.keys()))
    loaded_data.close()
else:
    print("Cannot examine array because the file was not loaded.")

## Visualize the Array
Use Matplotlib to create a visualization (e.g., a heatmap or image) of the `usgs_test_original` array. This is most effective for 2D arrays.

In [ ]:
# Check if the array exists and is suitable for visualization (e.g., 2D)
if 'usgs_test_original' in locals() and isinstance(usgs_test_original, np.ndarray):
    if usgs_test_original.ndim == 2:
        print("\nVisualizing the array as a heatmap...")
        plt.figure(figsize=(10, 8))
        plt.imshow(usgs_test_original, aspect='auto', cmap='viridis') # 'viridis' is a common colormap
        plt.colorbar(label='Value')
        plt.title('Visualization of usgs_test_original Array')
        plt.xlabel('Column Index')
        plt.ylabel('Row Index')
        plt.show()
    elif usgs_test_original.ndim == 1:
        print("\nVisualizing the array as a line plot...")
        plt.figure(figsize=(12, 4))
        plt.plot(usgs_test_original)
        plt.title('Visualization of usgs_test_original Array (1D)')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.show()
    else:
        print(f"\nArray has {usgs_test_original.ndim} dimensions. Visualization might not be straightforward.")
        print("Consider reshaping or selecting slices for visualization.")
else:
    print("\nCannot visualize because 'usgs_test_original' array is not available or not loaded correctly.")

# Inspect Processed Test Data (.npz)

This notebook loads a processed test data file (`.npz`) and examines the `usgs_test_original` array to understand its characteristics, particularly looking for zero values or other anomalies that might explain issues seen in evaluation plots.

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# --- Configuration ---
# Define the station ID to inspect
station_id = '21609641' # Options: '21609641', '20380357'

# Define paths relative to the notebook location
# Assuming the notebook is in the root of the project
PROJECT_ROOT = os.getcwd() # Get current working directory
PROCESSED_DATA_DIR = os.path.join(PROJECT_ROOT, 'data', 'processed')

file_path = os.path.join(PROCESSED_DATA_DIR, 'test', f"{station_id}.npz")

print(f"Looking for file at: {file_path}")

In [ ]:
# --- Load and Inspect Data ---
try:
    data = np.load(file_path, allow_pickle=True)
    print(f"Successfully loaded: {file_path}")
    print(f"\nAvailable keys in the file: {list(data.keys())}")

    if 'usgs_test_original' in data:
        usgs_original = data['usgs_test_original']
        print(f"\nShape of usgs_test_original: {usgs_original.shape}")

        # Convert to DataFrame for easier analysis
        # Columns represent lead times (1h to 18h)
        lead_times = [f"lead_{i}h" for i in range(1, usgs_original.shape[1] + 1)]
        usgs_df = pd.DataFrame(usgs_original, columns=lead_times)

        print("\nBasic statistics for usgs_test_original (each column is a lead time):")
        # Use display for better formatting in notebooks
        from IPython.display import display
        display(usgs_df.describe())

        # Check for zero values
        zero_counts = (usgs_df == 0).sum()
        print("\nCounts of zero values per lead time:")
        print(zero_counts)

        # Check for NaN values
        nan_counts = usgs_df.isna().sum()
        print("\nCounts of NaN values per lead time:")
        print(nan_counts)

        # Check for negative values
        negative_counts = (usgs_df < 0).sum()
        print("\nCounts of negative values per lead time:")
        print(negative_counts)

    else:
        print("\nError: Key 'usgs_test_original' not found in the file.")

except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

## Visualize Distribution

In [ ]:
# --- Visualize Data Distribution (Optional) ---
if 'usgs_df' in locals(): # Check if the DataFrame was created successfully
    print("\nGenerating boxplot of USGS original values across lead times...")
    plt.figure(figsize=(15, 7))
    sns.boxplot(data=usgs_df, showfliers=False) # showfliers=False matches evaluate.py
    plt.title(f"Distribution of usgs_test_original Values by Lead Time - Station {station_id}")
    plt.xlabel("Lead Time")
    plt.ylabel("USGS Flow Value (cms)")
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Plot histogram for a specific lead time (e.g., first lead time)
    first_lead_time_col = usgs_df.columns[0]
    print(f"\nGenerating histogram for the first lead time ({first_lead_time_col})...")
    plt.figure(figsize=(10, 5))
    sns.histplot(usgs_df[first_lead_time_col].dropna(), kde=True)
    plt.title(f"Histogram of USGS Flow for {first_lead_time_col} - Station {station_id}")
    plt.xlabel("USGS Flow Value (cms)")
    plt.ylabel("Frequency")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
else:
    print("\nSkipping visualization because usgs_df was not created (likely due to file load error).")